#### Tutorial
https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/


In [14]:
### Prerequiesites 

# pip install tensorflow==2.0
# pip install mtcnn
# pip install pillow
# pip install sklearn



In [1]:
import tensorflow as tf


model = tf.keras.models.load_model('models/keras/facenet_keras.h5')
print(model.inputs)
print(model.outputs)
model.summary()

[<tf.Tensor 'input_1:0' shape=(None, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/Identity:0' shape=(None, 128) dtype=float32>]
Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (

Total params: 22,808,144
Trainable params: 22,779,312
Non-trainable params: 28,832
__________________________________________________________________________________________________


In [15]:
from mtcnn import MTCNN
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def detect_face_reshape(path):
    bounding_box = MTCNN()
    mtcnn_input_size = (160, 160)
    
    # reading file 
    img = Image.open(path)
    ### NOTE: The image should be 3 dimensional or RGB format
    
    # converting to numpy array
    img_array =  np.asarray(img)
    
    # detecting face
    try:
        boundaries =  bounding_box.detect_faces(img_array)
        
        # getting detected variables
        x_1, y_1, w, h = boundaries[0]['box']
        x_2 = x_1 + w
        y_2 = y_1 + h
        
        focus_area = img_array[y_1:y_2, x_1:x_2]
        img_cropped = Image.fromarray(focus_area, 'RGB')
        
        img_cropped = img_cropped.resize(mtcnn_input_size)
        img_cropped_array = np.asarray(img_cropped)
#         plt.imshow(img_cropped_array)
#         plt.show()
        return img_cropped_array
    
    except Exception as e:
        print('Error while cropping: \n', e)
        
        
def normalize_tensor(img_array):
    mean = img_array.mean()
    std = img_array.std()
    img_array = (img_array - mean)/std
    
    return img_array

def process_all_face_data(folder_path):
    
    img_array_detected = []
    
    for file_name in os.listdir(folder_path):
        
        # training data
        # checking to read only jpeg files
        path = os.path.join(folder_path, file_name)
        img_cropped_array = detect_face_reshape(path)
        
        # normalize is correctly detected
        if img_cropped_array is not None:
            img_cropped_array =  normalize_tensor(img_cropped_array)
            img_array_detected.append(img_cropped_array)
            
    return img_array_detected

def preprocess_dataset(dir_path):
    personwise_faces = {}
    for i in os.listdir(dir_path):
        path = os.path.join(dir_path, i)
        personwise_faces[i] = process_all_face_data(path)
        
    return personwise_faces

In [16]:
def generate_X(personwise_faces):
    data = []
    for item in personwise_faces.keys():
        data.extend(personwise_faces[item])
    return data

def generate_y(personwise_faces):
    labels = []
    for item in personwise_faces.keys():
        labels.extend([item for i in range(len(personwise_faces[item]))])
        
    return labels


In [17]:
import os
cwd = os.getcwd()

train_path = os.path.join(cwd, 'dataset/5-celebrity-faces-dataset/train')
validation_path = os.path.join(cwd, 'dataset/5-celebrity-faces-dataset/val/')

# load train dataset
personwise_faces_train = preprocess_dataset(train_path)
X_train = generate_X(personwise_faces_train)
y_train = generate_y(personwise_faces_train)
print('len X:' , len(X_train), 'X shape:', X_train[0].shape, 'len y:', len(y_train))

# load train dataset
personwise_faces_valid = preprocess_dataset(validation_path)
X_val = generate_X(personwise_faces_valid)
y_val = generate_y(personwise_faces_valid)
print('len X:' , len(X_val), 'X shape:', X_val[0].shape, 'len y:', len(y_val))



Error while cropping: 
 Error when checking input: expected input_394 to have shape (None, None, 3) but got array with shape (180, 135, 4)
Error while cropping: 
 tile cannot extend outside image
len X: 91 X shape: (160, 160, 3) len y: 91
Error while cropping: 
 not enough values to unpack (expected 3, got 2)
len X: 24 X shape: (160, 160, 3) len y: 24


In [5]:
model_input = np.array(X_train)
print('Model input:', model_input.shape)

embeddings = model.predict(model_input)
print('Embeddings shape:', embeddings.shape)
print('Label shape:', len(y_train))

# Normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(embeddings)
X_train_ = scaler.transform(embeddings)



Model input: (91, 160, 160, 3)
Embeddings shape: (91, 128)
Label shape: 91


In [6]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train_, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [7]:
model_input = np.array(X_val)
print('Model input:', model_input.shape)

embeddings = model.predict(model_input)
print('Embeddings shape:', embeddings.shape)
print('Label shape:', len(y_val))

X_val_ = scaler.transform(embeddings)


Model input: (24, 160, 160, 3)
Embeddings shape: (24, 128)
Label shape: 24


In [8]:
y_pred = clf.predict(X_val_)

from sklearn.metrics import accuracy_score
print('Accuracy:', accuracy_score(y_val, y_pred)*100, '%')

Accuracy: 91.66666666666666 %


### Another Classifier

In [9]:
model_input = np.array(X_train)
print('Model input:', model_input.shape)

embeddings = model.predict(model_input)
print('Embeddings shape:', embeddings.shape)
print('Label shape:', len(y_train))

# Normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(embeddings)
X_train_ = scaler.transform(embeddings)



Model input: (91, 160, 160, 3)
Embeddings shape: (91, 128)
Label shape: 91


In [11]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier()
clf.fit(X_train_, y_train)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

In [12]:
model_input = np.array(X_val)
print('Model input:', model_input.shape)

embeddings = model.predict(model_input)
print('Embeddings shape:', embeddings.shape)
print('Label shape:', len(y_val))

X_val_ = scaler.transform(embeddings)


Model input: (24, 160, 160, 3)
Embeddings shape: (24, 128)
Label shape: 24


In [13]:
y_pred = clf.predict(X_val_)

from sklearn.metrics import accuracy_score
print('Accuracy:', accuracy_score(y_val, y_pred)*100, '%')

Accuracy: 100.0 %
